In [ ]:
import torch
import torch.nn as nn

from transformers import BertTokenizer, AutoTokenizer
from transformers import BertForSequenceClassification, AutoModelForSequenceClassification

from datasets import load_dataset
import numpy as np

from captum.attr import IntegratedGradients, LayerIntegratedGradients
from captum.attr import visualization

import shap
import scipy as sp

In [ ]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [ ]:
# XAI models needs to use a pre-trained model

# either train one your self on the IMDB dataset or find one pre-trained on huggingface!
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained("../models/bert_imdb.pt")

In [ ]:
class BertModelWrapper(nn.Module):
    
    def __init__(self, model):
        super(BertModelWrapper, self).__init__()
        self.model = model
        
    def forward(self, input_ids):        
        outputs = self.model(input_ids=input_ids)
        logits = outputs.logits
        return nn.functional.softmax(logits, dim=1)

In [ ]:
def input_ref(model_wrapper, sentence):
    input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True)], device=device)
    
    ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
    sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
    cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence
    
    ref_input_ids = [cls_token_id] + (input_ids.size(1)-2) * [ref_token_id] + [sep_token_id]
    ref_input_ids = torch.tensor([ref_input_ids], device=device)
    
    return input_ids, ref_input_ids

In [ ]:
def add_attributions_to_visualizer(attributions, tokens, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    #attributions = attributions / torch.norm(attributions)
    attributions = attributions.detach().cpu().clone().numpy()
    
    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions/np.linalg.norm(attributions),
                            pred,
                            pred_ind,
                            label,
                            "label",
                            attributions.sum(),       
                            tokens[:len(attributions)],
                            delta))    

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset, test_dataset = load_dataset('imdb', split=['train', 'test'])
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
bert_model_wrapper = BertModelWrapper(model)
lig = LayerIntegratedGradients(bert_model_wrapper, bert_model_wrapper.model.bert.embeddings)

# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

In [ ]:
model.to(device)
bert_model_wrapper.eval()
bert_model_wrapper.zero_grad()

In [ ]:
#r = np.random.choice(len(test_dataset))
r=10
label = test_dataset['label'][r].item()
sentence = test_dataset['text'][r]

In [ ]:
input_ids, ref_input_ids = input_ref(bert_model_wrapper, sentence)

pred = bert_model_wrapper(input_ids)[:, 1].unsqueeze(1).item()
pred_ind = round(pred)

attributions, delta = lig.attribute(inputs=input_ids, n_steps=500,
                                   baselines=ref_input_ids,
                                    internal_batch_size=32,
                                    return_convergence_delta=True,
                                    target=pred_ind
                                   )

In [ ]:
print('pred: ', pred_ind, '(', '%.2f' % pred, ')', ', delta: ', abs(delta))

tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().clone().numpy().tolist())    
add_attributions_to_visualizer(attributions, tokens, pred, pred_ind, label, delta, vis_data_records_ig)


In [ ]:
visualization.visualize_text(vis_data_records_ig)

In [ ]:
test_dataset['input_ids'].shape

In [ ]:
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=512,truncation=True) for v in x]).cuda()    
    attention_mask = (tv!=0).type(torch.int64).cuda()
    outputs = model(tv,attention_mask=attention_mask)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    return scores

In [ ]:
id2label = model.config.id2label
label2id = model.config.label2id
labels = sorted(label2id, key=label2id.get)

In [ ]:
explainer = shap.Explainer(f, tokenizer, output_names=labels)
shap_values = explainer([sentence])

In [ ]:
shap.plots.text(shap_values[:,:,'LABEL_1'])

In [ ]:
shap.plots.bar(shap_values[0,:,1])

In [ ]:
r = np.random.choice(len(test_dataset))
label = test_dataset['label'][r].item()
sentence = test_dataset['text'][r]

In [ ]:
sentence

In [ ]:
label

In [ ]:
shap_values = explainer([sentence])

In [ ]:
shap.plots.text(shap_values[:,:,0])